In [1]:
import pickle as pkl
import logging
import numpy as np
import gensim
import matplotlib.pyplot as plt
from gensim.models.ldamulticore import LdaMulticore
from gensim.models import CoherenceModel

In [2]:
# Switch for running new experiemnts vs. loading old ones


# load cleaned corpus
with open('data/data_lemmatized.pkl', 'rb') as f:
    data_lemmatized = pkl.load(f)
with open('data/cleaned_corpus.pkl', 'rb') as f:
    corpus = pkl.load(f)
with open("data/id2word.pkl", 'rb') as f:
    id2word= pkl.load(f)

In [1]:
# init storage

try_ntopics = np.array(list(range(2,42,2)))
num_ntopics = len(try_ntopics)

# save the list of numbers of topics to try
np.save("data/try_ntopics",try_ntopics)

NEW_EXPERIMENTS = False
perplexity = {}
coherence = {}
# loop through topic sizes

for i, topics in enumerate(try_ntopics):

    # enable logging
    #logging.basicConfig(filename=f'log_files/gensim_{topics}_topics_logscale.log',
     #               format="%(asctime)s:%(levelname)s:%(message)s",
     #               level=logging.INFO)

    # Build LDA model with this number of topics
    lda_model = LdaMulticore(corpus=corpus,
                             id2word=id2word,
                             num_topics=topics,
                             random_state=100,
                             chunksize=3000,
                             passes=1,
                             per_word_topics=True)

    #Compute Perplexity
    perplexity[topics] = lda_model.log_perplexity(corpus)  # a measure of how good the model is. lower the better.

    # Compute Coherence Score
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    coherence[topics] = coherence_model_lda.get_coherence()

    #save results
    #lda_model.save(f"trained_models/trained_lda_model_search_logscale_{topics}")
    with open("data/perplexity.pkl", 'wb') as f:
        pkl.dump(perplexity, f)
    with open("data/coherence.pkl", 'wb') as f:
        pkl.dump(coherence, f)         

    # screen report
    print(f"Num Topics = {topics}: Perplexity = {perplexity[topics]}, Coherence = {coherence[topics]}")

NameError: name 'np' is not defined

In [3]:
lda_model_x = LdaMulticore(corpus=corpus,
                         id2word=id2word,
                         num_topics=10,
                         random_state=100,
                         chunksize=3000,
                         passes=1,
                         per_word_topics=True)

In [4]:
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
vis =pyLDAvis.gensim_models.prepare(lda_model_x, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.026932 -0.017489       1        1  17.415023
2     -0.008404 -0.001054       2        1  15.598241
5      0.007645  0.042016       3        1  14.884551
3      0.027872  0.014944       4        1  11.506318
0     -0.046185 -0.006069       5        1   9.351676
9     -0.038140 -0.006953       6        1   8.291208
8      0.006862  0.019309       7        1   6.709503
1      0.016819 -0.010684       8        1   6.613705
7      0.060702 -0.028290       9        1   5.573757
6     -0.000239 -0.005730      10        1   4.056020, topic_info=         Term          Freq         Total Category  logprob  loglift
79      model  64028.000000  64028.000000  Default  30.0000  30.0000
118      soil  38333.000000  38333.000000  Default  29.0000  29.0000
125   surface  22115.000000  22115.000000  Default  28.0000  28.0000
136   wetland  15506.000000  15506.000000  Default  27.0000  27.0000
212     river  29390.000000  29390.000000  Default  26.0000  26.0000
..        ...           ...           ...      ...      ...      ...
125   surface    796.578486  22115.595283  Topic10  -5.4922  -0.1187
14     change    996.463974  36761.545457  Topic10  -5.2684  -0.4030
777      land    612.672930  14333.206396  Topic10  -5.7547   0.0525
186  increase    669.353895  20220.150147  Topic10  -5.6663  -0.2032
343   climate    690.685524  24329.813516  Topic10  -5.6349  -0.3568

[937 rows x 6 columns], token_table=       Topic      Freq  Term
term                        
40433      1  0.046372   abl
40433      2  0.092744   abl
40433      3  0.023186   abl
40433      4  0.046372   abl
40433      5  0.672391   abl
...      ...       ...   ...
479        6  0.103250  zone
479        7  0.074022  zone
479        8  0.066931  zone
479        9  0.034763  zone
479       10  0.053614  zone

[3665 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 3, 6, 4, 1, 10, 9, 2, 8, 7])